In [1]:
import graphlab as gl
import scipy as sp
import numpy as np
from HAMMER import train_xgboost, xgboost_predict

[WARNING] Unable to write current GraphLab Create license to /home/alvas/.graphlab/config. Ensure that this user account has write permission to /home/alvas/.graphlab/config to save the license for offline use.
[INFO] This non-commercial license of GraphLab Create is assigned to liling@coli.uni-saarland.de and will expire on October 11, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-3533 - Server binary: /usr/local/lib/python2.7/dist-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1454465611.log
[INFO] GraphLab Server Version: 1.8.1
[WARNING] Unable to create session in specified location: '/home/alvas/.graphlab/artifacts'. Using: '/var/tmp/graphlab-alvas/3533/tmp_session_01badd90-ad5f-4745-9dba-d0202b9fcc52'


In [2]:
sts_train = gl.SFrame('../sts2016_train.stasis/')
sts_test = gl.SFrame('../sts2016_test.stasis/')

In [3]:
sts_train = sts_train.dropna(columns=['Score'])

In [4]:
import graphlab
import numpy


train = sts_train

feat = [i for i in train.column_names() if i not in ['Dataset', 'Domain', 'Score', 'Sent1', 'Sent2']]

train_valid_shuffled = graphlab.toolkits.cross_validation.shuffle(train, random_seed=1)

def kth_start_end(n,k,kth):
    start = (n*kth)/k
    end = (n*(kth+1))/k-1
    return start, end+1

def get_kth_train(data, kth_start, kth_end, n):
    part1 = data[0:kth_start]
    part2 = data[kth_end+1:n]
    return part1.append(part2)

n = len(train)
accuracy = []
k = 10

In [5]:
feat = [i for i in train.column_names() if i not in ['Dataset', 'Domain', 'Score', 'Sent1', 'Sent2'] + \
        ['DLS_compose_ppmi', 'DLS_compose_cbow', 'prop_harmonic', 'glove_cosine']]

In [6]:


for i in range(k):
    start, end = kth_start_end(n,k,i)
    valid_kth = train_valid_shuffled[start:end]
    train_kth = get_kth_train(train_valid_shuffled, start, end, n)
    m = graphlab.boosted_trees_regression.create(train_kth, target='Score', features=feat, validation_set=None)
    print sp.stats.pearsonr(np.array(m.predict(valid_kth)), np.array(valid_kth['Score']))

PROGRESS: WARNING: Detected extremely low variance for feature(s) 'SP-Oc(UCP)', 'NE-Me(ANGLE_QUANTITY)', 'NE-Me(LANGUAGE)', 'NE-Me(MEASURE)', 'NE-Me(METHOD)', 'NE-Me(NUM)', 'NE-Me(PROJECT)', 'NE-Me(SIZE_QUANTITY)', 'NE-Me(SPEED_QUANTITY)', 'NE-Me(SYSTEM)', 'NE-Me(TEMPERATURE_QUANTITY)', 'NE-Oe(ANGLE_QUANTITY)', 'NE-Oe(LANGUAGE)', 'NE-Oe(MEASURE)', 'NE-Oe(METHOD)', 'NE-Oe(NUM)', 'NE-Oe(PROJECT)', 'NE-Oe(SPEED_QUANTITY)', 'NE-Oe(SYSTEM)', 'NE-Oe(TEMPERATURE_QUANTITY)' because all entries are nearly the same.
Proceeding with model training using all features. If the model does not provide results of adequate quality, exclude the above mentioned feature(s) from the input dataset.
PROGRESS: Boosted trees regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 12237
PROGRESS: Number of features          : 122
PROGRESS: Number of unpacked features : 122
PROGRESS: +-----------+--------------+--------------------+---------------+
PR

In [7]:
test2015 = train.filter_by('STS2015-gold', 'Dataset')
traintill2014 = train.filter_by(['STS2012-gold', 'STS2013-gold', 'STS2014-gold'], 'Dataset')

In [8]:
m1 = graphlab.boosted_trees_regression.create(traintill2014, target='Score', features=feat, validation_set=None)

PROGRESS: WARNING: Detected extremely low variance for feature(s) 'SP-Oc(UCP)', 'NE-Me(ANGLE_QUANTITY)', 'NE-Me(LANGUAGE)', 'NE-Me(MEASURE)', 'NE-Me(METHOD)', 'NE-Me(NUM)', 'NE-Me(PROJECT)', 'NE-Me(SIZE_QUANTITY)', 'NE-Me(SPEED_QUANTITY)', 'NE-Me(SYSTEM)', 'NE-Me(TEMPERATURE_QUANTITY)', 'NE-Oe(ANGLE_QUANTITY)', 'NE-Oe(LANGUAGE)', 'NE-Oe(MEASURE)', 'NE-Oe(METHOD)', 'NE-Oe(NUM)', 'NE-Oe(PROJECT)', 'NE-Oe(SIZE_QUANTITY)', 'NE-Oe(SPEED_QUANTITY)', 'NE-Oe(SYSTEM)', 'NE-Oe(TEMPERATURE_QUANTITY)' because all entries are nearly the same.
Proceeding with model training using all features. If the model does not provide results of adequate quality, exclude the above mentioned feature(s) from the input dataset.
PROGRESS: Boosted trees regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 8363
PROGRESS: Number of features          : 122
PROGRESS: Number of unpacked features : 122
PROGRESS: +-----------+--------------+-----------------

In [9]:
sp.stats.pearsonr(np.array(m1.predict(test2015)), np.array(test2015['Score']))

(0.76747146564404811, 0.0)

In [13]:
feat = [i for i in train.column_names() if i not in ['Dataset', 'Domain', 'Score', 'Sent1', 'Sent2'] + \
        ['DLS_compose_ppmi', 'DLS_cbow_ppmi', 'prop_harmonic', 'glove_cosine']]

bad_features = ['SP-Oc(UCP)', 'NE-Me(ANGLE_QUANTITY)', 'NE-Me(LANGUAGE)', 'NE-Me(MEASURE)', 
                'NE-Me(METHOD)', 'NE-Me(NUM)', 'NE-Me(PROJECT)', 'NE-Me(SPEED_QUANTITY)', 
                'NE-Me(SYSTEM)', 'NE-Me(TEMPERATURE_QUANTITY)', 'NE-Oe(ANGLE_QUANTITY)', 
                'NE-Oe(LANGUAGE)', 'NE-Oe(MEASURE)', 'NE-Oe(METHOD)', 'NE-Oe(NUM)', 
                'NE-Oe(PROJECT)', 'NE-Oe(SPEED_QUANTITY)', 'NE-Oe(SYSTEM)', 
                'NE-Oe(TEMPERATURE_QUANTITY)']

feat2 = [_f for _f in feat if _f not in bad_features]

m2 = graphlab.linear_regression.create(traintill2014, target='Score', features=feat2, validation_set=None)
sp.stats.pearsonr(np.array(m2.predict(test2015)), np.array(test2015['Score']))

PROGRESS: WARNING: Detected extremely low variance for feature(s) 'NE-Me(SIZE_QUANTITY)', 'NE-Oe(SIZE_QUANTITY)' because all entries are nearly the same.
Proceeding with model training using all features. If the model does not provide results of adequate quality, exclude the above mentioned feature(s) from the input dataset.
PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 8363
PROGRESS: Number of features          : 104
PROGRESS: Number of unpacked features : 104
PROGRESS: Number of coefficients    : 105
PROGRESS: Starting Newton Method
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | 1         | 2        | 0.0564

(0.80060123152467633, 0.0)

In [15]:
bad_features = ['NE-Me(SIZE_QUANTITY)', 'NE-Oe(SIZE_QUANTITY)']

feat3 = [_f for _f in feat2 if _f not in bad_features]
m3 = graphlab.linear_regression.create(traintill2014, target='Score', features=feat3, validation_set=None)
sp.stats.pearsonr(np.array(m3.predict(test2015)), np.array(test2015['Score']))


PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 8363
PROGRESS: Number of features          : 102
PROGRESS: Number of unpacked features : 102
PROGRESS: Number of coefficients    : 103
PROGRESS: Starting Newton Method
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | 1         | 2        | 0.056409     | 5.538508           | 0.952426      |
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: SUCCESS: Optimal solution found.
PROGRESS:


(0.80060123152467633, 0.0)

In [16]:
m3.coefficients.sort('value', ascending=False).print_rows(20)

+--------------------------+-------+----------------+-----------------+
|           name           | index |     value      |      stderr     |
+--------------------------+-------+----------------+-----------------+
|   NGRAM-cosChar4ngrams   |  None | 6.64541329135  |  1.38523937316  |
|   NGRAM-jacTok4ngrams    |  None | 6.06758408044  |  3.24606370942  |
|   NGRAM-jacChar3ngrams   |  None | 5.05746458861  |  1.06740806321  |
| NE-Me(DISTANCE_QUANTITY) |  None | 4.39902015316  |   1.8684383491  |
|   NGRAM-cosTok3ngrams    |  None | 3.51442526391  |  1.37728302729  |
|      METEOR-synonym      |  None | 3.15308000549  |  0.372721771383 |
|     DLS_compose_cbow     |  None | 2.89948587079  |  0.106539261168 |
|    METEOR-paraphrase     |  None | 2.65927743384  |  0.161514123733 |
|        METEOR-all        |  None |  2.6173040982  |  0.248316703592 |
|   NGRAM-jacTok2ngrams    |  None | 1.85492226175  |  1.01206294389  |
|       NE-Me(TIME)        |  None |  1.4508643107  |  0.6833207

In [40]:
import graphlab as gl
import scipy as sp
import numpy as np

In [46]:
m1_old = graphlab.boosted_trees_regression.create(traintill2014, target='Score', features=feat3, validation_set=None)
m3_old = graphlab.linear_regression.create(traintill2014, target='Score', features=feat3, validation_set=None)
print sp.stats.pearsonr(np.array(m1_old.predict(test2015)), np.array(test2015['Score']))
print sp.stats.pearsonr(np.array(m3_old.predict(test2015)), np.array(test2015['Score']))

PROGRESS: Boosted trees regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 8363
PROGRESS: Number of features          : 102
PROGRESS: Number of unpacked features : 102
PROGRESS: +-----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+--------------+--------------------+---------------+
PROGRESS: | 1         | 0.111682     | 4.01133            | 2.15363       |
PROGRESS: | 2         | 0.169839     | 3.87901            | 1.64403       |
PROGRESS: | 3         | 0.209472     | 3.66795            | 1.31346       |
PROGRESS: | 4         | 0.252653     | 3.62628            | 1.1061        |
PROGRESS: | 5         | 0.297478     | 3.63752            | 0.979447      |
PROGRESS: | 6         | 0.344821     | 3.64008            | 0.896498      |
PROGRESS: | 7         | 0.387687     | 3.67289            | 0.844464      |
PROG

In [53]:
train_valid_old = traintill2014[feat3+['Score']].to_dataframe()
gbm_old = train_xgboost(train_valid_old, feat3, num_boost_round=5000, eta=0.02, max_depth=8, test_size=0.2)
predictions_old = xgboost_predict(gbm_old, test2015[feat3].to_dataframe(), feat3)
print sp.stats.pearsonr(np.array(predictions_old), np.array(test2015['Score']))

Will train until eval error hasn't decreased in 100 rounds.
[0]	train-rmse:0.913444	eval-rmse:0.913698
[1]	train-rmse:0.896899	eval-rmse:0.897936
[2]	train-rmse:0.880650	eval-rmse:0.882264
[3]	train-rmse:0.864816	eval-rmse:0.867017
[4]	train-rmse:0.849292	eval-rmse:0.852042
[5]	train-rmse:0.834439	eval-rmse:0.837768
[6]	train-rmse:0.819591	eval-rmse:0.823518
[7]	train-rmse:0.805024	eval-rmse:0.809559
[8]	train-rmse:0.790751	eval-rmse:0.795930
[9]	train-rmse:0.776786	eval-rmse:0.782566
[10]	train-rmse:0.763067	eval-rmse:0.769620
[11]	train-rmse:0.749878	eval-rmse:0.757167
[12]	train-rmse:0.737139	eval-rmse:0.745018
[13]	train-rmse:0.724673	eval-rmse:0.733342
[14]	train-rmse:0.712128	eval-rmse:0.721542
[15]	train-rmse:0.699797	eval-rmse:0.709895
[16]	train-rmse:0.687786	eval-rmse:0.698649
[17]	train-rmse:0.676195	eval-rmse:0.687890
[18]	train-rmse:0.664571	eval-rmse:0.677157
[19]	train-rmse:0.653187	eval-rmse:0.666721
[20]	train-rmse:0.642350	eval-rmse:0.656604
[21]	train-rmse:0.631481	e

Validating
RMSE: 0.889242
(0.80741511314866665, 0.0)


[805]	train-rmse:0.069522	eval-rmse:0.283328
Stopping. Best iteration:
[705]	train-rmse:0.077948	eval-rmse:0.283247



In [54]:
m1 = graphlab.boosted_trees_regression.create(sts_train, target='Score', features=feat2, validation_set=None)

m3 = graphlab.linear_regression.create(sts_train, target='Score', features=feat3, validation_set=None)

PROGRESS: Boosted trees regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 13597
PROGRESS: Number of features          : 104
PROGRESS: Number of unpacked features : 104
PROGRESS: +-----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+--------------+--------------------+---------------+
PROGRESS: | 1         | 0.190662     | 4.30428            | 2.11016       |
PROGRESS: | 2         | 0.258727     | 4.14215            | 1.61343       |
PROGRESS: | 3         | 0.326030     | 4.14138            | 1.29205       |
PROGRESS: | 4         | 0.391642     | 3.86261            | 1.09126       |
PROGRESS: | 5         | 0.462069     | 3.83338            | 0.968893      |
PROGRESS: | 6         | 0.528827     | 3.87607            | 0.893777      |
PROGRESS: | 7         | 0.596910     | 3.8455             | 0.848049      |
PRO

In [55]:
train_valid = sts_train[feat3+['Score']].to_dataframe()
gbm = train_xgboost(train_valid, feat3, num_boost_round=5000, eta=0.02, max_depth=8, test_size=0.2)

Will train until eval error hasn't decreased in 100 rounds.
[0]	train-rmse:0.895630	eval-rmse:0.901163
[1]	train-rmse:0.879518	eval-rmse:0.885366
[2]	train-rmse:0.863749	eval-rmse:0.870009
[3]	train-rmse:0.848351	eval-rmse:0.855045
[4]	train-rmse:0.833195	eval-rmse:0.840375
[5]	train-rmse:0.818631	eval-rmse:0.826247
[6]	train-rmse:0.804404	eval-rmse:0.812432
[7]	train-rmse:0.790253	eval-rmse:0.798677
[8]	train-rmse:0.776276	eval-rmse:0.785222
[9]	train-rmse:0.762828	eval-rmse:0.772404
[10]	train-rmse:0.749707	eval-rmse:0.759851
[11]	train-rmse:0.737007	eval-rmse:0.747672
[12]	train-rmse:0.724451	eval-rmse:0.735693
[13]	train-rmse:0.711905	eval-rmse:0.723700
[14]	train-rmse:0.699669	eval-rmse:0.712023
[15]	train-rmse:0.687705	eval-rmse:0.700584
[16]	train-rmse:0.676168	eval-rmse:0.689530
[17]	train-rmse:0.664648	eval-rmse:0.678573
[18]	train-rmse:0.653622	eval-rmse:0.668138
[19]	train-rmse:0.642629	eval-rmse:0.657669
[20]	train-rmse:0.632102	eval-rmse:0.647650
[21]	train-rmse:0.621779	e

Validating
RMSE: 0.860477


Stopping. Best iteration:
[879]	train-rmse:0.092356	eval-rmse:0.280224



In [56]:
predictions = xgboost_predict(gbm, sts_test[feat2].to_dataframe(), feat3)

In [57]:
import os, io

def output_to_file(team_name, run_name, target):
    test_dir = '../sts2016-english-v1.1/'
    filenames = [test_dir+i for i in os.listdir(test_dir) if i.endswith('ascii')]
    for infile in filenames:
        domain = infile.rpartition('.')[0].rpartition('.')[2]
        outfile = 'STS2016.OUTPUT.'+team_name+'.'+run_name+'.'+domain+'.txt'
        with io.open(infile, 'r') as fin, io.open(outfile, 'w') as fout:
            domain_data = sts_test.filter_by(domain, column_name='Domain')
            for line, row in zip(fin, domain_data):
                #line = line.split('\t')
                #s1, s2 = line[0], line[1]
                fout.write(unicode(row[target])+'\n')
                
sts_test.add_column(gl.SArray(m1.predict(sts_test)), name='pred_m1')   
sts_test.add_column(gl.SArray(m3.predict(sts_test)), name='pred_m3')
sts_test.add_column(gl.SArray(predictions), name='pred_xgboost')
output_to_file('saarsheff', 'MT-Metrics-boostedtrees', 'pred_m1')
output_to_file('saarsheff', 'MT-Metrics-linear', 'pred_m3')
output_to_file('saarsheff', 'MT-Metrics-xgboost', 'pred_xgboost')
